In [ ]:
pip install flask huawei-lte-api
from flask import Flask, request, redirect, url_for
from huawei_lte_api.Client import Client
from huawei_lte_api.AuthorizedConnection import AuthorizedConnection
import requests

In [ ]:

app = Flask(__name__)

# MPESA API Credentials
MPESA_API_URL = "https://sandbox.safaricom.co.ke/mpesa/stkpush/v1/processrequest"
MPESA_CONSUMER_KEY = "XNhzEjP2xoFituRFUwz4WysuAmgWr1McFZABamNzjekRoyT3"
MPESA_CONSUMER_SECRET = "fYQDTEXNyky2FHuvYnplUXyCaAEFSKeyQ98xnCFGTwGWVKmGm0spCL1dApyjYo1h"
MPESA_PASSKEY = "your_mpesa_passkey"
MPESA_SHORTCODE = "your_shortcode"
MPESA_CALLBACK_URL = "your_callback_url"

# Router Credentials
ROUTER_IP = "192.168.8.1"
ROUTER_USERNAME = "admin"
ROUTER_PASSWORD = "sanity2022"

# Function to trigger MPESA payment
def initiate_mpesa_payment(phone, amount):
    headers = {
        "Authorization": "Bearer " + get_mpesa_access_token(),
        "Content-Type": "application/json"
    }

    payload = {
        "BusinessShortCode": MPESA_SHORTCODE,
        "Password": generate_mpesa_password(),
        "Timestamp": generate_timestamp(),
        "TransactionType": "CustomerPayBillOnline",
        "Amount": amount,
        "PartyA": phone,
        "PartyB": MPESA_SHORTCODE,
        "PhoneNumber": phone,
        "CallBackURL": MPESA_CALLBACK_URL,
        "AccountReference": "Wi-Fi Payment",
        "TransactionDesc": "Payment for Wi-Fi Access"
    }

    response = requests.post(MPESA_API_URL, json=payload, headers=headers)
    return response.json()

# Get MPESA access token
def get_mpesa_access_token():
    response = requests.get(
        "https://sandbox.safaricom.co.ke/oauth/v1/generate?grant_type=client_credentials",
        auth=(MPESA_CONSUMER_KEY, MPESA_CONSUMER_SECRET)
    )
    return response.json()["access_token"]

# Generate MPESA password
def generate_mpesa_password():
    import base64
    import datetime
    password = MPESA_SHORTCODE + MPESA_PASSKEY + generate_timestamp()
    password_bytes = password.encode('utf-8')
    password_encoded = base64.b64encode(password_bytes)
    return password_encoded.decode('utf-8')

# Generate timestamp
def generate_timestamp():
    from datetime import datetime
    return datetime.now().strftime("%Y%m%d%H%M%S")

# Function to allow access by adding MAC address to router
def allow_wifi_access(mac_address):
    connection = AuthorizedConnection(f'http://{ROUTER_USERNAME}:{ROUTER_PASSWORD}@{ROUTER_IP}/')
    client = Client(connection)
    data = {"mac_address": mac_address}
    response = client.device.signal()  # Just a placeholder; actual API may differ
    print(f"MAC {mac_address} allowed on the network.")

@app.route('/')
def index():
    return redirect(url_for('static', filename='pay.html'))

@app.route('/process_payment', methods=['POST'])
def process_payment():
    phone = request.form['phone']
    amount = request.form['amount']
    payment_response = initiate_mpesa_payment(phone, amount)
    
    if payment_response.get("ResponseCode") == "0":
        # Assume payment is successful, allow Wi-Fi access
        user_mac_address = get_user_mac(phone)
        allow_wifi_access(user_mac_address)
        return "Payment successful! Wi-Fi access granted."
    else:
        return f"Payment failed: {payment_response.get('errorMessage')}"

def get_user_mac(phone):
    # In real implementation, map phone number to MAC address
    return "AA:BB:CC:DD:EE:FF"

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)
